In [25]:
#coding=utf-8

In [26]:
import pandas as pd
import numpy as np

In [27]:
#given dataset and feature, return the entropy of subsets divided by the feature. If feature
#is "", return the entropy of the dataset itself
def entropy(data, feature):
    entropy = 0
    if feature != "":
        branches = list(set(data[feature]))
        for br in branches:
            br_data = data[data[feature] == br]
            br_length = len(br_data)
            p_yes = len(br_data[br_data.label == 'yes']) * 1.0 / br_length
            p_no = len(br_data[br_data.label == 'no']) * 1.0 / br_length
            p_yes_eff, p_no_eff = p_yes, p_no
            if p_yes == 0:
                p_yes_eff = 1
            if p_no == 0:
                p_no_eff = 1
            entropy += br_length * 1.0 / len(data) * (-p_yes * np.log(p_yes_eff) - p_no * np.log(p_no_eff))
        return entropy
    elif feature == "":
        p_yes = len(data[data.label == 'yes']) * 1.0 / len(data)
        p_no = len(data[data.label == 'no']) * 1.0 / len(data)
        p_yes_eff, p_no_eff = p_yes, p_no
        if p_yes == 0:
            p_yes_eff = 1
        if p_no == 0:
            p_no_eff = 1
        entropy = -p_yes * np.log(p_yes_eff) - p_no * np.log(p_no_eff)
        return entropy

In [30]:
def predict(conditions, model):
    for m in model:
        exit = True
        for key in m.keys():
            if key == 'void' or key == 'label':
                continue
            if m[key] != conditions[key]:
                exit = False
                break
        if exit:
            return m[key]

In [28]:
data = pd.DataFrame({"age": ['young', 'young', 'young', 'young', 'young', 'middle', 'middle', 'middle', 'middle', 'middle', 'old', 'old', 'old', 'old', 'old'], 
                     "haveWork": ['no', 'no', 'yes', 'yes', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'yes', 'yes', 'no'],
                     "haveHouse": ['no', 'no', 'no', 'yes', 'no', 'no', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'no', 'no', 'no'], 
                     "credit": ['normal', 'good', 'good', 'normal', 'normal', 'normal', 'good', 'good', 'very good', 'very good', 'very good', 
                               'good', 'good', 'very good', 'normal'], 
                     "label": ['no', 'no', 'yes', 'yes', 'no', 'no', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'no']})
features = ['age', 'credit', 'haveHouse', 'haveWork']
data

,age,credit,haveHouse,haveWork,label
0,young,normal,no,no,no
1,young,good,no,no,no
2,young,good,no,yes,yes
3,young,normal,yes,yes,yes
4,young,normal,no,no,no
5,middle,normal,no,no,no
6,middle,good,no,no,no
7,middle,good,yes,yes,yes
8,middle,very good,yes,no,yes
9,middle,very good,yes,no,yes


In [29]:
#the main flow
model = []
paths = [{"void": ""}] #each path is a boolean with multiple conditions
datasets = [data] #the data subsets at the bottom of the tree
critia = 0.001 #if the decrease ratio of entropy is blow this value, stop dividing
while True:
    datasets_new = []
    paths_new = []
    gain = 0
    hasUpdate = False
    for n, i in enumerate(datasets):
        #print '\n'
        #print 'dataset: ', i
        #select the feature that produces most info gain on each subset
        entropy_base = entropy(data = i, feature = "")
        if entropy_base == 0: #all the labels are the same
            datasets_new.append(i)
            paths_new.append(paths[n])
            continue
        for m, j in enumerate(features):
            #print 'feature: ', j
            if m == 0:
                entropy_m = entropy(data = i, feature = j)
                gain = abs(entropy_base - entropy_m) / entropy_m
                feature = j
                #print 'gain: ', gain
            elif m > 0:
                entropy_m = entropy(data = i, feature = j)
                gain_m = abs(entropy_base - entropy_m) / entropy_m
                if gain_m > gain:
                    gain = gain_m
                    feature = j
                #print 'gain: ', gain
        #update the active datasets            
        if gain > critia:
            feature_keys = list(set(i[feature]))
            for key in feature_keys:
                datasets_new.append(i[i[feature] == key]) #split based on the possible values of the chosen feature
                path = paths[n].copy()
                path[feature] = key
                paths_new.append(path)
                #print 'update path: '
                #print paths[n]
                #print path
            hasUpdate = True
            #print '>>>>>>>>>>>>split on ' + feature
        elif gain <= critia:
            datasets_new.append(i)
            paths_new.append(paths[n])
    if hasUpdate:
        datasets = datasets_new
        paths = paths_new
    elif not hasUpdate:
        break
for n, i in enumerate(paths):
    i['label'] = list(datasets[n]['label'])[0]
    model.append(i)
print 'model: '
print model

model: 
[{'void': '', 'haveHouse': 'yes', 'label': 'yes'}, {'haveWork': 'yes', 'void': '', 'haveHouse': 'no', 'label': 'yes'}, {'haveWork': 'no', 'void': '', 'haveHouse': 'no', 'label': 'no'}]


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in double_scalars


In [37]:
print predict(conditions = {'haveHouse': 'no', 'haveWork': 'yes', 'credit': 'bad'}, model = model)

no
